# OpenStack_WordCount

In [ ]:
import pandas as pd
from nltk.corpus import stopwords 
import nltk
from nltk.corpus import treebank
import nltk.stem 
from nltk import stem
import numpy as np
import string
import re
import random
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
#Randomly select 1000 from bugslist and report (launch pad and stack over flow)

In [ ]:
filename = "buglist.xlsx"
df_launchpad=pd.read_excel(filename)
df=pd.DataFrame(df_launchpad)
del(df_launchpad)
df.set_index=df.bug
df=df.drop(['project','component','version','fault_class','fault_type','fault_symptom','severity','priority','status','mitigation','log','repro','submitter','assignee','created','deployment'], axis=1)

In [ ]:

filename = "ask_os.csv"
df_ask_os = pd.read_csv(filename)
df_ask=pd.DataFrame(df_ask_os)
del(df_ask_os)
df_ask.set_index=df_ask.url
df_ask=df_ask.drop(['Serial','answer'],axis=1)


In [ ]:
filename = "report.xlsx"
df_stackoverflow = pd.read_excel(filename)

df_st=pd.DataFrame(df_stackoverflow)
del(df_stackoverflow)
df_st.set_index=df_st.id
df_st=df_st.drop(['answers','link','Creator','date','votes','fetch_url'],axis=1)


In [ ]:
stop_words=stopwords.words('english')
stop_word_dict=[]
for i in stop_words:
    stop_word_dict.append(str(i.encode("utf-8")))
common_words=['',"aren't",'ffffffffff']
stop_word_dict=stop_word_dict + common_words

In [ ]:
def create_dict(dataframe,field,stop_word_dict):
    corpus=[]
    for i in field:
        
        if type(i)== int or type(i)== float:  
            continue
        i = re.sub(r'[^\x00-\x7f]','',i)
        for k in i:
            if type(k) == type(u''):
                k = k.encode("utf-8",'ignore')
        corpus.append(str(i).strip())
    dataframe['field_corpus']=pd.Series(corpus)
    dataframe['field_corpus']=pd.Series( str(i).lower()  if len(str(i)) >0 else '' for i in dataframe.field_corpus)
    corpus=[]
    for i in dataframe.field_corpus:
        data=(str(i))
        data=re.sub(r'[^\x00-\x7f]','',data) or re.sub(r'[0-9]','',data)
        corpus.append(data) 
    dataframe.field_corpus_new=pd.Series(corpus)
    corpus=[]
    new_corpus=[]
    row_corpus=[]
    for i in dataframe.field_corpus_new:
        substr=re.split('[^A-Za-z]',i)
        for j in substr:
            j=j.strip()
            if j not in stop_word_dict and len(j)>1:
                corpus.append(j)   
        new_corpus.append(corpus)
        corpus=[]
    dataframe.field_words_bag=pd.Series(new_corpus)
    field_words_count={}
    for i in new_corpus:
        for w in i:
            field_words_count[w]=field_words_count.get(w,0)+1 
    return field_words_count,dataframe.field_words_bag

In [ ]:
my_words_count={}

In [ ]:

Fault_Details_lp= pd.Series(df.fault_description+df.req)
#df=df.drop(['fault_description','req'],axis=1)
df=df.assign(Fault_Details_lp=Fault_Details_lp.values)
launchpad_words_count={}
launchpad_words_count,df['field_words_bag']=create_dict(df,df.Fault_Details_lp,stop_word_dict)
#df=df.drop(['Fault_Details_lp'],axis=1)
for k,v in launchpad_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(launchpad_words_count)


In [ ]:
len(my_words_count)

In [ ]:

Fault_Details_st= pd.Series(df_st.title+df_st.tags+ df_st.longtext.to_string())
#df_st=df_st.drop(['tags','title','longtext'],axis=1)
df_st=df_st.assign(Fault_Details_st=Fault_Details_st.values)
stackoverflow_words_count={}
stackoverflow_words_count,df_st['field_words_bag']=create_dict(df_st,df_st.Fault_Details_st,stop_word_dict)
#df_st=df_st.drop(['Fault_Details_st'],axis=1)
for k,v in stackoverflow_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(stackoverflow_words_count)



In [ ]:
df_st.head()

In [ ]:
len(my_words_count)

In [ ]:
Fault_Details_ask = pd.Series(df_ask.summary+df_ask.title)
#df_ask=df_ask.drop(['summary','title'],axis=1)
df_ask=df_ask.assign(Fault_Details_ask=Fault_Details_ask.values)
ask_words_count={}
ask_words_count,df_ask['field_words_bag']=create_dict(df_ask,df_ask.Fault_Details_ask,stop_word_dict)
#df_ask=df_ask.drop(['Fault_Details_ask'],axis=1)
for k,v in ask_words_count.items():
    my_words_count[k]=my_words_count.get(k,1)+v
del(ask_words_count)

In [ ]:
df_ask.head()

In [ ]:
len(my_words_count)


In [ ]:
#Remove non-ascii character to process data using NLTK
#NLTK coverts string into meaningful words called token, removes punctuations and special characters.
#Analyzing Fault_desc and Req field from data set
#write process data to csv file

In [ ]:
#launchpad_words_count={}
#launchpad_words_count,df['field_words_bag']=create_dict(df,df.Fault_Details_lp,stop_word_dict)

In [ ]:
mylist=[]
for key, value in sorted(my_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
df_wordcount=pd.DataFrame(mylist, columns =['Key','Count'])
#del (mylist)

In [ ]:
#del (my_words_count)


In [ ]:
len(df_wordcount)

In [ ]:
df_wordcount.to_csv('dictionary.csv')

In [ ]:
print "dictionary created"

In [ ]:
mydict=set(df_wordcount.Key)

In [ ]:
del (df_wordcount)

<div>
    <a href="http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html" target="_blank" title="nlp.stanford.edu" style="display: block; text-align: center;"><img src="http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html" alt="nlp.stanford.edu" style="max-width: 100%;width: 800px;"  width="800"/></a>
</div>

In [ ]:
#Using TF-IDF to convert unstructured text to useful features
'''All_dict={}
for i in range(0,len(df_wordcount)):
    All_dict[df_wordcount.Key[i]]=df_wordcount.Count[i]'''

In [ ]:
'''def computeTF(All_dict, field):
    tfDict = {}
    fieldCount= len(field)
    for key, value in All_dict.iteritems():
        tfDict[key] = value/ float(fieldCount)
    return tfDict'''

In [ ]:
'''tf_Fault_Details_lp = computeTF(All_dict,df.Fault_Details_lp)
tf_Fault_Details_st = computeTF(All_dict,df_st.Fault_Details_st)'''

In [ ]:
'''def computeIDF(docList):
    import math
    idfDict= {}
    N= len(docList)
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.iteritems():
            if val > 0:
                idfDict[word]+=1
    for word,val in idfDict.iteritems():
        idfDict[word] = math.log(N/ float(val))
    return idfDict'''

In [ ]:
'''idfs_lp=computeIDF([tf_Fault_Details_lp])
idfs_st=computeIDF([tf_Fault_Details_st])'''

In [ ]:
'''idfs_lp'''

In [ ]:
'''def computeTFIDF(tfList, idfs):
    tfidf={}
    for word,val in tfList.iteritems():
        tfidf[word]= val *  idfs[word]
    return tfidf'''

In [ ]:
'''tfidf_launchpad_list = computeTFIDF(launchpad_words_count, idfs_lp)
tfidf_stackoverflow_list = computeTF(stackoverflow_words_count, idfs_st)
'''

In [ ]:
'''heading=['Launch_pad','Stack_overflow']
tfMatrix=pd.DataFrame([tfidf_launchpad_list,tfidf_stackoverflow_list])
Temp=pd.Series(heading)
tfMatrix= tfMatrix.assign(Temp=Temp.values)
tfMatrix.index=Temp
tfMatrix.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\tfmatrix - Copy.csv')
tfMatrix.head()'''

# Using NLTK

In [ ]:
#from nltk.stem.snowball import SnowballStemmer
#stemmer = SnowballStemmer("english")

In [ ]:
'''def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens'''

In [ ]:
'''from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df.Fault_Details_lp)

print(tfidf_matrix.shape)'''

In [ ]:
'''terms = tfidf_vectorizer.get_feature_names()'''

In [ ]:
'''from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)'''

# K-means clustering

In [ ]:
'''from sklearn.cluster import KMeans
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()'''

In [ ]:
'''from sklearn.externals import joblib
joblib.dump(km,  'C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\LaundPad.pkl')'''

In [ ]:
'''km = joblib.load('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\LaundPad.pkl')
clusters = km.labels_.tolist()'''

In [ ]:
'''import pandas as pd
Cluster=pd.Series(clusters)
df=df.assign(Cluster=Cluster.values)'''

In [ ]:
'''df.head()'''

In [ ]:
'''df.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\Classified_LaunchPad.csv')'''

# DTM

In [ ]:
#Create DTM for corpus

In [ ]:
'''from sklearn.feature_extraction.text import CountVectorizer'''

In [ ]:
'''vocab=mydict'''

In [ ]:
'''simple_vectorizer = CountVectorizer(vocabulary=vocab,min_df=0., max_df=1.0)
bow1 = simple_vectorizer.fit_transform(df.Fault_Details_lp)
bow2 = simple_vectorizer.fit_transform(df_st.Fault_Details_st)
bow3 = simple_vectorizer.fit_transform(df_ask.Fault_Details_ask)
bow1_df=pd.DataFrame(bow1.A, columns=simple_vectorizer.get_feature_names())
bow2_df =pd.DataFrame(bow2.A,columns=simple_vectorizer.get_feature_names())
bow3_df =pd.DataFrame(bow3.A,columns=simple_vectorizer.get_feature_names())
'''



In [ ]:
'''words = list(simple_vectorizer.vocabulary_.keys())'''

In [ ]:
#Concatenate DTM with original dataset using index

In [ ]:
'''result1 = pd.concat([df, bow1_df], axis=1, join='inner')
result2 = pd.concat([df_st, bow2_df], axis=1, join='inner')
result3=  pd.concat([df_ask, bow3_df], axis=1, join='inner')'''

In [ ]:
'''result1.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\df_expand1 - Copy.csv',encoding="utf-8")
df_st_expand.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\df_st_expand2 - Copy.csv',encoding="utf-8")'''

In [ ]:
'''result2.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\df_expand2 - Copy.csv',encoding="utf-8")'''

In [ ]:
'''result3.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\df_expand3 - Copy.csv',encoding="utf-8")'''

In [ ]:
#Topic Modeling: This script loads a gensim dictionary and associated corpus and applies an LDA model.

In [ ]:
from gensim import corpora, models
import pyLDAvis.gensim
from optparse import OptionParser

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
#build Dictinoary

In [ ]:
opts= "open_stack"
dictionary = corpora.Dictionary([mydict])
dictionary.compactify()

In [ ]:
print("Dictionary: ")
print(dictionary)

In [ ]:
#Save dictionary for future use

In [ ]:
del(mydict)

In [ ]:
dictionary.save("open_stack.dict")

In [ ]:
#build Corpus

In [ ]:
corpus =[dictionary.doc2bow(df.field_words_bag.loc[i]) for i in range(0,len(df))] 
corpus= corpus + [dictionary.doc2bow(df_st.field_words_bag.loc[i]) for i in range(0,len(df_st))] 
corpus= corpus +[dictionary.doc2bow(df_ask.field_words_bag.loc[i]) for i in range(0,len(df_ask))]

In [ ]:
print(corpus)

In [ ]:
#Save corpus for future use

In [ ]:
corpora.MmCorpus.serialize("open_stack.mm", corpus)

In [ ]:
# Load the corpus and Dictionary

In [ ]:
corpus = corpora.MmCorpus("open_stack.mm")
dictionary = corpora.Dictionary.load("open_stack.dict")

In [ ]:
# Initialize Parameters

In [ ]:
num_topics,passes,alpha=5,20,0.001

In [ ]:
#Apply LDA


In [ ]:
lda = models.LdaModel(corpus, id2word=dictionary,
                        num_topics=num_topics,
                        passes=passes,
                        alpha =alpha)

In [ ]:
#Save lda results for future reference

In [ ]:
lda.save("open_stack.lda")

In [ ]:
lda.print_topics()

In [ ]:
print(lda)

In [ ]:
# First LDA model with 5 topics, 20 passes, alpha = 0.001

In [ ]:
openstack_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(openstack_data)

In [ ]:
pyLDAvis.save_html(openstack_data,"openstack.html")
#pyLDAvis.save_html(vis_data,outpth+'LDA_Visualization.html')

In [ ]:
token_table=pd.DataFrame(openstack_data.token_table )
token_table.to_csv("open_stack_token_table.csv")